# Retrievers

In [6]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

In [7]:
loaders = [
    TextLoader("data/langchain.md"),
    TextLoader("data/langchain2.md"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Define a text splitter that will be used to create child documents from larger parent documents.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# Initialize a vector store named "full_documents" which will index the child chunks of the documents.
# The OllamaEmbeddings model "snowflake-arctic-embed:33m" is used to generate embeddings for these chunks.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# Set up an in-memory storage layer that will store the parent documents.
store = InMemoryStore()

# Create a retriever that uses the previously defined vector store, document store, and child splitter.
# This retriever will be able to fetch relevant parent documents based on queries and split them into child chunks as needed.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [ ]:
retriever.add_documents(docs, ids=None)
list(store.yield_keys())

In [ ]:
sub_docs = vectorstore.similarity_search("LangChian")


In [ ]:
retrieved_docs = retriever.invoke("LangChian")
len(retrieved_docs[0].page_content)


15683

## Retrieving larger chunks

In [14]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# The storage layer for the parent documents
store = InMemoryStore()

In [ ]:
# Create a retriever that uses the previously defined vector store, document store, child splitter, and parent splitter.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)


In [ ]:
retriever.add_documents(docs)
len(list(store.yield_keys()))


29

In [ ]:
sub_docs = vectorstore.similarity_search("LangChian")

print(sub_docs[0].page_content)

```
### Streaming[​](https://python.langchain.com/docs/tutorials/summarization/#streaming "Direct link to Streaming")


In [ ]:
retrieved_docs = retriever.invoke("LangChian")

len(retrieved_docs[0].page_content)
print(retrieved_docs[0].page_content)

*   128k token OpenAI `gpt-4o`
*   200k token Anthropic `claude-3-5-sonnet-20240620`

The chain will take a list of documents, insert them all into a prompt, and pass that prompt to an LLM:

```
from langchain.chains.combine_documents import create_stuff_documents_chainfrom langchain.chains.llm import LLMChainfrom langchain_core.prompts import ChatPromptTemplate# Define promptprompt = ChatPromptTemplate.from_messages(    [("system", "Write a concise summary of the following:\\n\\n{context}")])# Instantiate chainchain = create_stuff_documents_chain(llm, prompt)# Invoke chainresult = chain.invoke({"context": docs})print(result)
```
### Streaming[​](https://python.langchain.com/docs/tutorials/summarization/#streaming "Direct link to Streaming")

Note that we can also stream the result token-by-token:

```
for token in chain.stream({"context": docs}):    print(token, end="|")
```



### Go deeper[​](https://python.langchain.com/docs/tutorials/summarization/#go-deeper "Direct link to Go dee